In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [2]:
# loading the data
with open('../Final_Intents.json', 'r') as file:
    data = json.load(file)

In [3]:
# Function to pair each question with its corresponding response
def pair_questions_responses(data):
    paired_data = []
    for item in data:
        tag = item.get('tag', 'Unknown')
        questions = item.get('questions', [])
        responses = item.get('responses', [])
        
        for question, response in zip(questions, responses):
            paired_data.append({'tag': tag, 'question': question, 'response': response})
    
    return paired_data

In [4]:
# Use the function to pair questions and responses
paired_data = pair_questions_responses(data)

# Now, 'paired_data' contains each question paired with its corresponding response


In [5]:
number_to_display = 5  # You can change this to display more or fewer pairs

for i, pair in enumerate(paired_data[:number_to_display]):
    print(f"Pair {i + 1}")
    print("Tag:", pair['tag'])
    print("Question:", pair['question'])
    print("Response:", pair['response'])
    print("-" * 50)  # Separator for readability

Pair 1
Tag: cloud_computing
Question: What is the fundamental starting point for understanding Cloud Computing?
Response: The fundamental starting point for understanding Cloud Computing is to recognize it as a pivotal element in modern IT, enabling the delivery of computing services over the internet.
--------------------------------------------------
Pair 2
Tag: cloud_computing
Question: How does Moringa School introduce the concept of Cloud Computing?
Response: Moringa School introduces the concept of Cloud Computing through a comprehensive program that covers key principles, services, and practical applications.
--------------------------------------------------
Pair 3
Tag: cloud_computing
Question: Why is Cloud Computing considered essential in the real world?
Response: Cloud Computing is considered essential in the real world because it provides scalable, cost-effective solutions for organizations, enabling them to innovate and streamline operations.
-----------------------------

In [6]:
df = pd.DataFrame(paired_data)
df.head(20)

,tag,question,response
0,cloud_computing,What is the fundamental starting point for und...,The fundamental starting point for understandi...
1,cloud_computing,How does Moringa School introduce the concept ...,Moringa School introduces the concept of Cloud...
2,cloud_computing,Why is Cloud Computing considered essential in...,Cloud Computing is considered essential in the...
3,cloud_computing,Can you provide an overview of the Cloud Compu...,The Cloud Computing program at Moringa School ...
4,cloud_computing,What are the eligibility requirements for the ...,To be eligible for the Cloud Computing program...
5,cloud_computing,How long do students need to dedicate to succe...,Students are required to dedicate 3 days to le...
6,cloud_computing,What unique learning experiences does Moringa ...,Moringa School offers a unique learning experi...
7,cloud_computing,How is the curriculum at Moringa School tailor...,The curriculum at Moringa School is tailored t...
8,cloud_computing,What specific areas does the Cloud Computing c...,The Cloud Computing course covers various area...
9,cloud_computing,In what ways does Moringa School guarantee a c...,Moringa School guarantees a comprehensive unde...


In [7]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tag       599 non-null    object
 1   question  599 non-null    object
 2   response  599 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
questions = [d['question'] for d in paired_data]
responses = [d['response'] for d in paired_data]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + responses)

# Convert texts to sequences
questions_seq = tokenizer.texts_to_sequences(questions)
responses_seq = tokenizer.texts_to_sequences(responses)

# Finding the maximum sequence length
max_length_questions = max(len(seq) for seq in questions_seq)
max_length_responses = max(len(seq) for seq in responses_seq)
max_seq_length = max(max_length_questions, max_length_responses)

# Padding sequences
questions_padded = pad_sequences(questions_seq, maxlen=max_seq_length, padding='post')
responses_padded = pad_sequences(responses_seq, maxlen=max_seq_length, padding='post')

# One-hot encode the responses
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
responses_one_hot = np.zeros((len(responses_padded), max_seq_length, vocab_size))
for i, sequence in enumerate(responses_padded):
    for j, word_index in enumerate(sequence):
        responses_one_hot[i, j, word_index] = 1

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

embedding_dim = 100  # Embedding dimensionality
latent_dim = 256  # Latent dimensionality of the LSTM layers

# Encoder
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_seq_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 208)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 208)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 208, 100)             191000    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 208, 100)             191000    ['input_2[0][0]']             
                                                                                            

In [11]:
# Train the model
model.fit([questions_padded, responses_padded], responses_one_hot,
          batch_size=64,
          epochs=10,
          validation_split=0.2)

Epoch 1/10


8/8 [==============================] - 37s 4s/step - loss: 4.9259 - accuracy: 0.7703 - val_loss: 0.8913 - val_accuracy: 0.9140
Epoch 2/10
8/8 [==============================] - 24s 3s/step - loss: 0.9980 - accuracy: 0.8901 - val_loss: 0.7227 - val_accuracy: 0.9140
Epoch 3/10
8/8 [==============================] - 24s 3s/step - loss: 0.9282 - accuracy: 0.8901 - val_loss: 0.7027 - val_accuracy: 0.9140
Epoch 4/10
8/8 [==============================] - 24s 3s/step - loss: 0.9047 - accuracy: 0.8902 - val_loss: 0.6825 - val_accuracy: 0.9140
Epoch 5/10
8/8 [==============================] - 24s 3s/step - loss: 0.9151 - accuracy: 0.8903 - val_loss: 0.6743 - val_accuracy: 0.9140
Epoch 6/10
8/8 [==============================] - 24s 3s/step - loss: 0.8843 - accuracy: 0.8902 - val_loss: 0.6591 - val_accuracy: 0.9140
Epoch 7/10
8/8 [==============================] - 24s 3s/step - loss: 0.9273 - accuracy: 0.8905 - val_loss: 0.6740 - val_accuracy: 0.9142
Epoch 8/10
8/8 [================

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

# Predicting on test data
test_pred_raw = model.predict([test_questions_padded, test_responses_padded])
test_pred = np.argmax(test_pred_raw, axis=-1)
true_test_labels = np.argmax(test_responses_one_hot, axis=-1)

# Flatten the predictions and true values
flat_test_pred = test_pred.flatten()
flat_true_test_labels = true_test_labels.flatten()

# Generating a classification report
print(classification_report(flat_true_test_labels, flat_test_pred))

# Generating a confusion matrix
conf_matrix = confusion_matrix(flat_true_test_labels, flat_test_pred)
print(conf_matrix)

NameError: name 'test_questions_padded' is not defined

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate([test_questions_padded, test_responses_padded], test_responses_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Visualize some predictions (for text data)
for i in range(5):  # Display 5 predictions
    print("Question:", seq_to_text(test_questions_padded[i]))
    print("True Response:", seq_to_text(test_responses_padded[i]))
    print("Predicted Response:", predicted_texts[i])
    print("-" * 50)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming binary classification and you have binary labels
fpr, tpr, thresholds = roc_curve(flat_true_test_labels, flat_test_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have history object from model.fit()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()